# ML Project

Implementing Mogrifier LSTM model in tensorflow.

In [ ]:
import numpy as np
import unicodedata
import tensorflow as tf
import gensim
from tensorflow import keras
from sklearn.model_selection import train_test_split

import nltk
nltk.download('brown')
from nltk.corpus import brown

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [ ]:
def strip_accents(s):
   return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')

# Mogrifier + LSTM

In [ ]:
def weight(units , factorize_k = None):
    if factorize_k is None:
        return keras.layers.Dense(units , activation = None , use_bias = False)
    
    assert factorize_k < units

    return keras.models.Sequential([
        keras.layers.Dense(factorize_k , activation = None , use_bias = False),
        keras.layers.Dense(units , activation = None , use_bias = False)
    ])

class MogrifierLSTM(keras.layers.LSTMCell):
    def __init__(self, units , iters = 5 , factorize_k = None , **kwargs):
        self.iters = iters
        self.factorize_k = factorize_k

        self.Q = weight(units , factorize_k)
        self.R = weight(units , factorize_k) if iters > 1 else None

        super().__init__(units , **kwargs)

    def call(self, input_at_t , state_at_t, **kwargs):

        if input_at_t is not None and state_at_t is not None:

          shape = input_at_t.shape
          *_ , units = shape

          h = state_at_t[0]
          c = state_at_t[1]

          x = tf.reshape(input_at_t , (-1 , units))
          h = tf.reshape(h , (-1 , units))

          for ind in range(self.iters):
              if(ind % 2) != 0:
                  x = 2 * tf.sigmoid(self.Q(h)) * x
              else:
                  h = 2 * tf.sigmoid(self.Q(x)) * h
          
          input_at_t = tf.reshape(x , shape)
          state_at_t = (tf.reshape(h , shape) , c)

        return super().call(inputs = input_at_t , states = state_at_t)

# Brown Dataset

In [ ]:
sentences = brown.sents()
model = gensim.models.Word2Vec(sentences, min_count=1)
dataset = [np.array([model[j] for j in i]) for i in sentences]
dataset = np.array(dataset)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  after removing the cwd from sys.path.


# Model

In [ ]:
myCell = MogrifierLSTM(100)
RNNlayer = keras.layers.RNN(myCell)

## Training Function

In [ ]:
def train(dataset, loss_fn , opt , EPOCHS):
  for i in range(EPOCHS):
    losses = []
    for datapoint in dataset:
      if(datapoint.shape[0] > 1):
        x_train = datapoint[0:-1 , :]
        y_true = (datapoint[-1 , :]).reshape((1 ,-1))
        # print(x_train.shape , y_true.shape)
        with tf.GradientTape() as tape:
          y_pred = RNNlayer(x_train.reshape((1 , -1 , 100)))
          loss = loss_fn(y_true , y_pred)
        print(loss.numpy())
        grads = tape.gradient(loss , tape.watched_variables())
        opt.apply_gradients(zip(grads, tape.watched_variables()))
        losses.append(loss.numpy())

## Testing Function

In [ ]:
def test(dataset, loss_fn):
  for datapoint in dataset:
    if(datapoint.shape[0] > 1):
      x_train = datapoint[0:-1 , :]
      y_true = (datapoint[-1 , :]).reshape((1 ,-1))
      # print(x_train.shape , y_true.shape)
      y_pred = RNNlayer(x_train.reshape((1 , -1 , 100)))
      loss = loss_fn(y_true , y_pred)
      print(loss.numpy())

## Model Parameters

In [ ]:
loss_fn = lambda y_true , y_pred : 1 - keras.losses.CosineSimilarity()(y_true , y_pred)
optimizer = keras.optimizers.Adam()
train_ds , test_ds = train_test_split(dataset , test_size = 0.2 , random_state = 25)
print('Training Dataset: ' , train_ds.shape )
print('Testing Dataset: ' , test_ds.shape)

Training Dataset:  (45872,)
Testing Dataset:  (11468,)


## Training

In [ ]:
train(train_ds , loss_fn , optimizer , 2);

Streaming output truncated to the last 5000 lines.
0.00017422438
0.00067567825
6.586313e-05
0.00012212992
0.3409896
8.493662e-05
0.00035578012
0.000988245
0.00014126301
0.00025838614
0.01843834
0.017873585
0.00013262033
0.0002451539
0.00022685528
9.6440315e-05
0.001132071
0.00024598837
0.00021708012
0.00024414062
0.00019985437
0.013215601
0.017165542
8.7201595e-05
0.7574431
0.00021666288
0.00015252829
0.00023496151
0.00019204617
0.4038393
0.006620705
0.0009814501
0.016925693
0.0009663105
0.029048026
0.0011744499
0.00055748224
0.00065642595
0.0011529922
0.0011113882
0.0016540289
0.0008572936
0.0005429387
0.0011035204
0.0011091232
0.00030618906
0.0006277561
0.03849864
0.00050902367
0.00022274256
0.000613153
0.00016456842
0.0005147457
0.37815195
0.00038260221
0.00046986341
0.00018334389
0.08777261
0.0016098022
0.00040388107
0.00031602383
0.0004892945
0.00033676624
0.00050342083
0.016875803
0.0011550188
0.00026488304
0.00025051832
0.003629446
0.00028657913
0.00029534101
0.00020855665
0.000

## Testing

In [ ]:
test(test_ds , loss_fn);